In [0]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10

In [1]:
cd drive/My Drive

[Errno 2] No such file or directory: 'drive/My Drive'
/content


In [0]:
from keras.models import Model
import keras.layers
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate
from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
import numpy as np
seed = 7
np.random.seed(seed)
 
# 卷积层和标准层组成的block
def Conv2d_BN(x, nb_filter,kernel_size, padding='same',strides=(1,1),name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
 
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)
    x = BatchNormalization(axis=3,name=bn_name)(x)
    return x
 
# Inception Block
def Inception(x,nb_filter):
    branch1x1 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)
 
    branch3x3 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)
    branch3x3 = Conv2d_BN(branch3x3,nb_filter,(3,3), padding='same',strides=(1,1),name=None)
 
    branch5x5 = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),name=None)
    branch5x5 = Conv2d_BN(branch5x5,nb_filter,(1,1), padding='same',strides=(1,1),name=None)
 
    branchpool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
    branchpool = Conv2d_BN(branchpool,nb_filter,(1,1),padding='same',strides=(1,1),name=None)
 
    x = concatenate([branch1x1,branch3x3,branch5x5,branchpool],axis=3)
 
    return x
 
inpt = Input(shape=(32,32,3))
#padding = 'same'，填充为(步长-1）/2,还可以用ZeroPadding2D((3,3))


x = Conv2d_BN(inpt,64,(3,3),strides=(1,1),padding='same')
x = Conv2d_BN(x,192,(3,3),strides=(1,1),padding='same')
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
x = Inception(x,64)#256
x = Inception(x,120)#480
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
x = Inception(x,128)#512
x = Inception(x,128)
x = Inception(x,128)
x = Inception(x,132)#528
x = Inception(x,208)#832
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
x = Inception(x,208)
x = Inception(x,256)#1024
x = AveragePooling2D(pool_size=(4,4),strides=(4,4),padding='same')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(1000,activation='relu')(x)
x = Dense(10,activation='softmax')(x)
model = Model(inpt,x,name='inception')
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.summary()

Model: "inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 32, 32, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 64)   256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 32, 32, 192)  110784      batch_normalization_57[0][0]     
__________________________________________________________________________________________

In [0]:


from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D, AveragePooling2D,Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler


Using TensorFlow backend.


In [0]:
# -*- coding: utf-8 -*-
import pickle as p
import numpy as np
import os

num_class = 10
def load_CIFAR_batch(filename):
    """ 载入cifar数据集的一个batch """
    with open(filename, 'rb') as f:
        datadict = p.load(f, encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,3,32,32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        Y = np_utils.to_categorical(Y, num_classes)
        
        return X, Y


def load_CIFAR10(ROOT):
    """ 载入cifar全部数据 """
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b,))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)         #将所有batch整合起来
        ys.append(Y)
    Xtr = np.concatenate(xs) #使变成行向量,最终Xtr的尺寸为(50000,32,32,3)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()
    
    # Resize training images
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_valid = np_utils.to_categorical(Y_valid, num_classes)
    
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')

    # preprocess data
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0

    return X_train, Y_train, X_valid, Y_valid

In [0]:
cifar10_dir = ''
x_train, y_train,x_test, y_test = load_CIFAR10(cifar10_dir)


In [0]:
print(x_train.shape)

(50000, 32, 32, 3)


In [0]:
# initiate RMSprop optimizer
#opt = keras.optimizers.Adam(lr=0.0001)

# Let's train the model using RMSprop
#model.compile(loss='categorical_crossentropy',
                            #optimizer=opt,
                            #metrics=['accuracy'])
from keras.models import load_model, Model
model = load_model("googlenet3.h5")
print("train____________")
model.fit(x_train,y_train,epochs=100,batch_size=128,)
print("test_____________")
loss,acc=model.evaluate(x_test,y_test)
print("loss=",loss )
print("accuracy=",acc)
model.save('googlenet4.h5')


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
train____________
Epoch 1/100
50000/50000 [==============================] - 218s 4ms/step - loss: 0.0022 - acc: 0.9992
Epoch 2/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0031 - acc: 0.9991
Epoch 3/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0016 - acc: 0.9995
Epoch 4/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0023 - acc: 0.9992
Epoch 5/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0032 - acc: 0.9989
Epoch 6/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0038 - acc: 0.9989
Epoch 7/100
50000/50000 [==============================] - 192s 4ms/step - loss: 0.0027 - acc: 0.9991
Epoch 8/100
50000/50000 [==============================] 

In [0]:
model.summary()

Model: "inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 32, 32, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 64)   256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 32, 32, 192)  110784      batch_normalization_57[0][0]     
__________________________________________________________________________________________